In [1]:
from roboflow import Roboflow
from ultralytics import YOLO  # Import the YOLO class from the ultralytics package


In [26]:

# Initialize the model for segmentation
# Replace 'yolov8n-seg.pt' with the specific model you intend to use, e.g., 'yolov8s-seg.pt', 'yolov8m-seg.pt', etc.
model = YOLO('yolov8s-seg.pt')  

model.train(data='demo3-1/data.yaml', epochs=50, project='models')  

# Evaluate the model's performance on the validation set
metrics = model.val()  
print(metrics)


New https://pypi.org/project/ultralytics/8.1.30 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.24 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M3 Pro)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=demo3-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/demo3-1/train/labels.cache... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]
val: Scanning /Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/demo3-1/valid/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]

Plotting labels to models/train4/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to models/train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      3.414      5.083      6.575      1.477        320        640: 100%|██████████| 2/2 [00:21<00:00, 10.83s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]

...


50 epochs completed in 0.284 hours.
Optimizer stripped from models/train4/weights/last.pt, 23.8MB
Optimizer stripped from models/train4/weights/best.pt, 23.8MB

Validating models/train4/weights/best.pt...
Ultralytics YOLOv8.1.24 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M3 Pro)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


                   all          5        334       0.57      0.437      0.469      0.217      0.383      0.335      0.316     0.0897
Speed: 1.3ms preprocess, 172.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to models/train4
Ultralytics YOLOv8.1.24 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M3 Pro)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


val: Scanning /Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/demo3-1/valid/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


                   all          5        334       0.57      0.437      0.469      0.217      0.383      0.335      0.316     0.0897
Speed: 1.2ms preprocess, 169.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to models/train42
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x2dbc49490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023

In [28]:
import matplotlib.pyplot as plt  # Import matplotlib for plotting
from PIL import Image  # For loading images

# Load the trained model (replace 'yolov8n-seg.pt' with your model's weight file)
# model = YOLO('models/yolov8s_trained_weights.pt')  # Use the path to your trained weights
model = YOLO('/Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/models/train4/weights/best.pt') 
# Load the first image from your validation dataset
# Replace 'path/to/first/validation/image.jpg' with the path to the actual image
image_path = '/Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/demo3-1/test/images/S08_02_SE1_1000X99_png.rf.053590ae637b3f347e0398ef6f654718.jpg'
image2_path = '/Users/sprice/Documents/GitHub/satellite_v2/conferenceDemoTest/demo3-1/train/images/S04_03_SE1_1000X53_png.rf.2f2a18cf111b493923e35151663798b4.jpg'

image = Image.open(image_path)
image2 = Image.open(image2_path)

# Perform prediction (detection)
results = model(image2)

annotated_image = results[0].plot()

if not isinstance(annotated_image, Image.Image):
    annotated_image = Image.fromarray(annotated_image)
    

# Display the annotated image
plt.figure(figsize=(26, 20))  # Adjust the figure size as needed
plt.imshow(annotated_image)
plt.axis('off')  # Hide axes ticks
plt.savefig(f'output.png')





0: 480x640 23 Satellites, 156.0ms
Speed: 2.3ms preprocess, 156.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)


In [140]:
import numpy as np
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.morphology import convex_hull_image
import math

def fractal_dimension(Z, threshold=0.9):
    # Only for 2d image, Z is binary image
    assert(len(Z.shape) == 2)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                           np.arange(0, Z.shape[1], k), axis=1)
        return len(np.where((S > 0) & (S < k*k))[0])
    # Transform Z into a binary array
    Z = (Z < threshold)
    p = min(Z.shape)
    n = 2**np.floor(np.log(p)/np.log(2))
    n = int(np.log(n)/np.log(2))
    sizes = 2**np.arange(n, 1, -1)
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]


masks = [mask.xy for idx, mask in enumerate(results[0].masks)]
for mask in masks:
    x_coords = mask[0][:, 0]
    y_coords = mask[0][:, 1]

    polygon_mask = np.zeros((int(max(y_coords)) + 1, int(max(x_coords)) + 1), dtype=np.uint8)
    rr, cc = polygon(y_coords, x_coords)
    polygon_mask[rr, cc] = 1
    props = regionprops(polygon_mask)

    for prop in props:
        print("Area:", prop.area)
        print("Perimeter:", prop.perimeter)
        print("Eccentricity:", prop.eccentricity)
        print('Roundness:', (4 * prop.area) / (np.pi * (prop.major_axis_length ** 2)))
        print("Equivalent Diameter:", prop.equivalent_diameter)
        print('Feret Diameter:', prop.feret_diameter_max)
        print("Centroid Location:", prop.centroid)
        print("Major Axis:", prop.axis_major_length)
        print("Minor Axis:", prop.axis_minor_length)
        print('Elongation:', prop.minor_axis_length / prop.major_axis_length)
        print('Crofton Perimeter:', prop.perimeter_crofton)
        print('Solidity:', prop.solidity)
        print('Convex Area:',prop.area_convex)
        print('Extent:', prop.extent)
        print('Convexity:', prop.perimeter / np.sum(np.sqrt(convex_hull_image(prop.coords))))
        print('Aspect Ratio:', prop.major_axis_length / prop.minor_axis_length)
        print('Fractal Dimension:', fractal_dimension(prop.image))
        print('Form Factor:', 4 * math.pi * prop.area / (prop.perimeter ** 2))
        print('Rectangularity:', prop.area / prop.area_bbox)
        print('Compactness:', (prop.perimeter ** 2) / (4 * math.pi * prop.area))
        print('Shape Factor:', math.sqrt(prop.area) / prop.perimeter)
        print("Convex Area Ratio:", prop.area / prop.convex_area)
        x_center, y_center = (prop.bbox[0] + prop.bbox[2]) / 2.0, (prop.bbox[1] + prop.bbox[3]) / 2.0
        centroidToCenter = math.sqrt((prop.centroid[0] - x_center) ** 2 + (prop.centroid[1] - y_center) ** 2)
        print("Centroid To Center:", centroidToCenter)
        print('-'*30)








# # Display the polygon mask
# plt.imshow(polygon_mask, cmap='gray')
# plt.title("Polygon Mask")
# plt.savefig('test.png')


Area: 9401.0
Perimeter: 411.5807358037436
Eccentricity: 0.8161431438322478
Roundness: 0.5476774318059879
Equivalent Diameter: 109.40623821361953
Feret Diameter: 153.37861650177967
Centroid Location: (361.7716200404212, 334.2726305712158)
Major Axis: 147.83581607611782
Minor Axis: 85.42689386344671
Elongation: 0.5778497804582216
Crofton Perimeter: 392.88452265348957
Solidity: 0.9481593545133636
Convex Area: 9915.0
Extent: 0.7852489141329769
Convexity: 0.021892592329986364
Aspect Ratio: 1.7305535691421792
Fractal Dimension: 1.0058658002289285
Form Factor: 0.6973869852112937
Rectangularity: 0.7852489141329769
Compactness: 1.4339240926571362
Shape Factor: 0.235576511928006
Convex Area Ratio: 0.9481593545133636
Centroid To Center: 6.463476004781073
------------------------------
Area: 7836.0
Perimeter: 356.06601717798213
Eccentricity: 0.6557172149265622
Roundness: 0.7290534242434824
Equivalent Diameter: 99.88545976539696
Feret Diameter: 125.60254774486066
Centroid Location: (294.93415007656

In [49]:
import cv2
image_paths = [image_path, image2_path]
images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]  # Convert images to RGB format

# Process the batch of images
results = model(images)

# results.xyxy contains bounding box coordinates, confidences and class labels
# results.render() will draw the predictions on the images
# Note: Each entry in results.xyxy corresponds to one input image
for i, (img, pred) in enumerate(zip(images, results)):
    # Convert bounding boxes from tensor to numpy array and render them on the image
    annotated_image = results[i].plot()  # Rendered image with detections
    if not isinstance(annotated_image, Image.Image):
        annotated_image = Image.fromarray(annotated_image)
        
    # Display the annotated image
    plt.figure()  # Adjust the figure size as needed
    plt.imshow(annotated_image)
    plt.axis('off')  # Hide axes ticks
    plt.savefig(f'output_{i}.png')


0: 640x640 7 Powders, 1: 640x640 146 Powders, 220.2ms
Speed: 3.1ms preprocess, 110.1ms inference, 20.0ms postprocess per image at shape (1, 3, 640, 640)
